# Lamda functions


# Process:
 
1.Presign s3 url 
2.load_files_version1 (MyStateMachine)
3.whisper_transcription_version1 (MyStateMachine)
4.summarization_version1 (MyStateMachine)
5.results_TranscriptionAI_Version1 (MyStateMachine)



## 1. Presign s3 url 

External layers: N/A

import json
import boto3
import os
import base64
from time import gmtime, strftime
from decimal import Decimal

  

def lambda_handler(event, context):
    
    print (event)
    
    base64_body=event["body"]
   
    print(f"base64_body is: {base64_body}")
    body_base64_bytes = base64_body.encode('ascii')
    body_bytes = base64.b64decode(body_base64_bytes)
    body_str = body_bytes.decode('ascii')
    
    body_dict = json.loads(body_str)
    
    print(f"message is {body_dict}")
    print(type(body_dict))
    
    file_name=body_dict["file_name"]
    file_size_int=int(body_dict["file_size"])/1000
    file_size_dec = json.loads(json.dumps(file_size_int), parse_float=Decimal)
    print(file_size_dec)
    email=body_dict["email"]
    
    path_to_save_file= email+"/"+file_name
   
   
    #store this request to dynamo DB
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('###')
    
    # store the current time in a human readable format in a variable
    date= strftime("%d %b %Y")
    time= strftime(" %R ")
    
    response2= table.put_item(
        Item={
            "ID":file_name,
            "email":email,
            "file_path":path_to_save_file,
            "file_size":file_size_dec,
            "upload_date":date,
            "upload_time":time
            }
    )
   
   
    #request presign url 
    s3 = boto3.client("s3")
    
    bucketName = os.environ.get("bucketName")
    print(bucketName)

    object_name=event["body"]
    
    
    print(f"file mame is:{object_name}")    
    
    expiration=300
    
    response=s3.generate_presigned_post(
        bucketName,
        path_to_save_file,
        ExpiresIn=expiration
    )
    
    
    
    print(response)
    
    #return must contain headers:"Access-Control-Allow-Origin":"*", to allow for any origin to read response.
    return {
        'statusCode': 200,
        'headers':{
            "Access-Control-Allow-Origin":"*"
        },
        'body': json.dumps(response)
    }


## 2. start_StateMachine

External layers: N/A

import json
import base64
import boto3

def lambda_handler(event, context):
    
    #1. print invpke from s3
    '''
    
    input=
        {
          "Records": [
            {
              "eventVersion": "2.0",
              "eventSource": "aws:s3",
              "awsRegion": "us-east-1",
              "eventTime": "1970-01-01T00:00:00.000Z",
              "eventName": "ObjectCreated:Put",
              "userIdentity": {
                "principalId": "EXAMPLE"
              },
              "requestParameters": {
                "sourceIPAddress": "127.0.0.1"
              },
              "responseElements": {
                "x-amz-request-id": "EXAMPLE123456789",
                "x-amz-id-2": "EXAMPLE123/5678abcdefghijklambdaisawesome/mnopqrstuvwxyzABCDEFGH"
              },
              "s3": {
                "s3SchemaVersion": "1.0",
                "configurationId": "testConfigRule",
                "bucket": {
                  "name": "example-bucket",
                  "ownerIdentity": {
                    "principalId": "EXAMPLE"
                  },
                  "arn": "arn:aws:s3:::example-bucket"
                },
                "object": {
                  "key": "test%2Fkey",
                  "size": 1024,
                  "eTag": "0123456789abcdef0123456789abcdef",
                  "sequencer": "0A1B2C3D4E5F678901"
                }
              }
            }
          ]
        }
    
    '''
   
    print(event)
    
    #2. start statemachine execution
    s3 = boto3.client("s3")
    client = boto3.client('stepfunctions')
    response = client.start_execution(
        stateMachineArn = 'arn:aws:states:us-east-1:###',
        input = json.dumps(event)
        )
    
    
    
    return {
        'statusCode': 200,
        'body': "End of  start_StateMachine function!"
    }


## 3. load_files_version1: Load media files, convert and create chunks according to requirements of Whisper API

External layers required:

ffmpeg: ffmpeg_dowload_johnvansickle_statictar: binary file "python.zip/python/ffmpeg/ffmpeg 

ffprobre: ffprobe_dowload_johnvansickle_statictar_v2:  binary file "python.zip/python/ffprobe/ffprobe

import json
import boto3
import urllib.parse # library to decode @ on email texts
from pathlib import Path #library to extract name of the file
import os
import math

import ffmpeg
import ffprobe
import subprocess
from subprocess import call



def lambda_handler(event, context):
    
    #Layers---------------------------------------------------------------#
    print("#Layers test---------------------------------------------------#")
    
    print(f"\n FFMPEG: {os.system('/opt/python/ffmpeg/ffmpeg')}")
    print(f"\n FFPROBE: {os.system('/opt/python/ffprobe/ffprobe')}")
   
    
    print("#End Layer test section-----------------------------------------#")
    #------------------------------------------------------------------------#
    
    '''
    
    input=
        {
          "Records": [
            {
              "eventVersion": "2.0",
              "eventSource": "aws:s3",
              "awsRegion": "us-east-1",
              "eventTime": "1970-01-01T00:00:00.000Z",
              "eventName": "ObjectCreated:Put",
              "userIdentity": {
                "principalId": "EXAMPLE"
              },
              "requestParameters": {
                "sourceIPAddress": "127.0.0.1"
              },
              "responseElements": {
                "x-amz-request-id": "EXAMPLE123456789",
                "x-amz-id-2": "EXAMPLE123/5678abcdefghijklambdaisawesome/mnopqrstuvwxyzABCDEFGH"
              },
              "s3": {
                "s3SchemaVersion": "1.0",
                "configurationId": "testConfigRule",
                "bucket": {
                  "name": "example-bucket",
                  "ownerIdentity": {
                    "principalId": "EXAMPLE"
                  },
                  "arn": "arn:aws:s3:::example-bucket"
                },
                "object": {
                  "key": "test%2Fkey",
                  "size": 1024,
                  "eTag": "0123456789abcdef0123456789abcdef",
                  "sequencer": "0A1B2C3D4E5F678901"
                }
              }
            }
          ]
        }
    
    '''
   
    print(event)
    
    input=event
    
    #1. get the input information and organize it
    print("#1. get the input information and organize it")
    
    #get bucket name from input
    bucket=input["Records"][0]["s3"]["bucket"]["name"]
    
    #get keys names from input and stored as list. First decode the key to add '@'
    input_key= input["Records"][0]["s3"]["object"]["key"]
    decoded_key = urllib.parse.unquote(input_key)
    decoded_key_with_spaces=decoded_key.replace("+", " ")
    keys=[]
    keys.append(decoded_key_with_spaces)

    print(f"bucket is: {bucket}")
    print(f"keys are: {keys}")
    
    #delete all files in temporal lambda folder
    call('rm -rf /tmp/*', shell=True)
    
    #identify media keys.
    media_extensions = ('Opus','Vorbis','mp2','mp3','mp4','m4a','mpga','mpg','mpeg','mov','LC-AAC','HE-AAC','WMAv1','WMAv2','AC3','E-AC3','mkv','mka','3gp','3g2','ps','m2ts','rmvb','ogg','flv','f4v','wav','webm')
    media_keys=[]
    
    for key in keys:
        extension_file=key.split(".")[-1] 
        if extension_file in media_extensions:
            media_keys.append(key)
        
    print(f"Media files found in bucket are {media_keys}")
    

    #2. transcode media files to .mp3 if necesary for supported formats in whisper and download temporally media file
    print("#2. transcode media files to .mp3 if necesary for supported formats in whisper and download temporally media file")
    
    whisper_supported_formats = ('m4a','mp3','webm','mp4','wav','mpeg')
    supported_media_files=[]
    
    for media_key in media_keys:
        extension_file=media_key.split(".")[-1]
        original_solo_email=key.split("/")[0] #get the original solo email to be used when uploading chunks files
        original_solo_file=key.split("/")[1] #get the original solo video filename for lambda local saved path
        
        
        if extension_file in whisper_supported_formats:
            
            
            # downloading file to /tmp directory within lambda
            lambda_file_path = f"/tmp/{original_solo_file}"
            s3 = boto3.client("s3")
            s3.download_file(
                Bucket=bucket,
                Key=media_key, 
                Filename=lambda_file_path)
                
            supported_media_files.append(original_solo_file)
        
            
        else:
            #upload file to .mp3 format
            original_extension=media_key.split(".")[-1] 
            file_name=media_key.replace(f".{original_extension}","")
            temporal_filename=f"{file_name}.mp3"
            print(f"temporal_filename :{temporal_filename}")
            
            #extract the email and filename to work with on local lambda
            temp_email=temporal_filename.split("/")[0]
            temp_filename=temporal_filename.split("/")[1]
            print(f"temp_email :{temp_email}")
            print(f"temp_filename :{temp_filename}")
            
            # path to download file to /tmp directory within lambda
            lambda_file_path = f"/tmp/{original_solo_file}"
            s3 = boto3.client("s3")
            s3.download_file(
                Bucket=bucket,
                Key=media_key, 
                Filename=lambda_file_path)
            
            #path to saved the trascode file
            lambda_output_file_path = f"/tmp/{temp_filename}"
            
            # transcode video/audio file to .mp3
            os.system(
                f"/opt/python/ffmpeg/ffmpeg -i {lambda_file_path} {lambda_output_file_path}"
            )

            # uploading transcoded file
            s3.upload_file(
                Bucket=bucket,
                Key=temporal_filename,
                Filename=lambda_output_file_path
            )
            supported_media_files.append(temp_filename)
            
            #eliminate the original downloaded file from the temporal lambda folder, leaving only the transcoding file
            call(f"rm -rf {lambda_file_path}", shell=True)
    
    
    print(f"supported media files are: {supported_media_files}")
    
    temporal_files=os.listdir('/tmp')
    print(f"There are {len(temporal_files)} files in temporal folder: {temporal_files}")
    
    
    #3. create chunks to fit whisper api requirements (25MB each) and upload to s3
    print("#3. create chunks to fit whisper api requirements (50MB each) and upload to s3")
    
    #select file to work on, assuming theres only one. Array[0]
    file_to_slice=supported_media_files[0]
    print(f"The file to split in chunks is {file_to_slice}")
    
    lambda_file_to_slice_path=f"/tmp/{file_to_slice}"
    
    #get the number of chunks, based of 25Mb limit of whisper API
    file_size=float(os.path.getsize(lambda_file_to_slice_path)/1000000)
    print(f"File size is {file_size} Mb")
    chunks_number=math.ceil(file_size/20) #defines 20Mb to goes safe
    print(F"{chunks_number} chunks are required to process the file in whisper due to the 25Mb limit")
    
    #get the duration of the file in seconds
    out = subprocess.check_output(["/opt/python/ffprobe/ffprobe", "-v", "quiet", "-show_format", "-print_format", "json", lambda_file_to_slice_path])
    ffprobe_data = json.loads(out)
    #print(ffprobe_data)
    duration_seconds = float(ffprobe_data["format"]["duration"])
    print(F"duration of the file is {duration_seconds} seconds")
    
    #calculates each chunk lenght in seconds (round down "marh.ceil" to minimize errors)
    chunk_lenght=math.ceil(duration_seconds/chunks_number)
    print(F"Each chunk lenght is {chunk_lenght} seconds")
    
    
    created_keys=[]
    
    if chunks_number>1:
    
        #define the format name for chunks
        file_name=Path(file_to_slice).stem
        extension_file=file_to_slice.split(".")[-1]
        lambda_file_to_slice_path_output=f"/tmp/{file_name}"
        
        #creates chunks using this ffmpeg function
        os.system(
            f"/opt/python/ffmpeg/ffmpeg -i '{lambda_file_to_slice_path}' -c copy -map 0 -segment_time {chunk_lenght} -f segment '{lambda_file_to_slice_path_output}'_%d.{extension_file}"
        )
        
        #list all files in temporal lambda folder
        temporal_files=os.listdir('/tmp/')
        print(f"There are {len(temporal_files)} files in temporal folder: {temporal_files}")
        
        #eliminate the original downloaded file from the temporal lambda folder
        call(f"rm -rf {lambda_file_to_slice_path}", shell=True)
        temporal_files=os.listdir('/tmp')
        print(f"There are {len(temporal_files)} files in temporal folder: {temporal_files}")
        
        #shows the chunks created
        chunks_keys=temporal_files
        if file_to_slice in chunks_keys:
            chunks_keys.remove(file_to_slice) #remove the converted file from the chunks list
        print(f"There were {len(chunks_keys)} chunks created: {chunks_keys}")
        
        
        #upload chunks to s3
        s3 = boto3.client("s3")
        chunks_keys_s3_path=[]
        for i in range (len(chunks_keys)):
            
            #add email folderupfront of upload path
            chunks_keys_s3_path.append(f"{original_solo_email}/{chunks_keys[i]}")
            
            lambda_file_to_upload_path = f"/tmp/{chunks_keys[i]}"
            s3.upload_file(
                    Bucket=bucket,
                    Key=chunks_keys_s3_path[i],
                    Filename=lambda_file_to_upload_path,
                )
        
    else: 
       chunks_keys=file_to_slice #no chunks were created. Output file name is the file_to_slice
       chunks_keys_s3_path=f"{original_solo_email}/{chunks_keys}"
    
    
    return {
        'statusCode': 200,
        'body': "End of load files function!",
        'email':original_solo_email,
        'input_file':original_solo_file,
        'working_file':file_to_slice,
        'chunks_keys':chunks_keys_s3_path,
        'chunks_keys_qty':int(chunks_number)
    }

## 4. Whisper_transcription_version1: transcribe audio files and upload transcript to s3

External layers required:

Openai: openai_installed_vscode_openai0.26_urllib3 1.26:  
      
    pip install --target=C:\Users\priet\projects\aws\packages\openai openai urllib3==1.26
    reasons:https://community.openai.com/t/openai-cannot-import-name-default-ciphers-problem-breaking-changes-with-the-latest-version-of-requests/194265   

import json
import boto3
from pathlib import Path #library to extract name of the file
import os
import openai


def lambda_handler(event, context):
    
    #Layers---------------------------------------------------------------#
    print("#Layers test---------------------------------------------------#")
    
    print(f"openai version is: {openai.__version__}")
   
    print("#End Layer test section-----------------------------------------#")
    #------------------------------------------------------------------------#
    
    '''
    input={
        
          "statusCode": 200,
          "body": "\"Hello from Lambda!\"",
          "email": "###",
          "input_file": "###,
          "working_file": "###",
          "chunks_keys": [
            "comite de inversiones - aug 2022_1.mp4",
            "comite de inversiones - aug 2022_5.mp4",
            "comite de inversiones - aug 2022_7.mp4",
            "comite de inversiones - aug 2022_0.mp4",
            "comite de inversiones - aug 2022_6.mp4",
            "comite de inversiones - aug 2022_4.mp4",
            "comite de inversiones - aug 2022_2.mp4",
            "comite de inversiones - aug 2022_3.mp4"
          ]
          "chunks_keys_qty": 8
        }
        
    '''
    
    input=event
    
    print(f"input : {input}")
    
    email=input["email"]
    
    #main inputs. chunks are array to sort in order
    working_file=input["working_file"]
    chunks_keys=[]
    chunks_keys_qty=int(input["chunks_keys_qty"])
    
    #organize chunks in ascending order
    
    print(f"chunks_keys_qty:{chunks_keys_qty}")
    
  
    
    if chunks_keys_qty>1:
        chunks_keys=list(input["chunks_keys"])
        chunks_keys.sort()
    
    else:
        chunks_keys.append(input["chunks_keys"])
        
    print(chunks_keys)
    
    bucket="transcription.version1"
    
    #1. download chunks files temporally in lambda folder
    print("#1. download chunks files temporally in lambda folder")
  
    openai.api_key=os.environ['OPENAI_API_KEY']
    
    text=""
    s3 = boto3.client("s3")
    for chunk in chunks_keys:
        
        print(f"\nProcessing file {chunk}")
        
        
        # downloading file to /tmp directory within lambda
        original_solo_file=chunk.split("/")[1] #get the original solo video filename for lambda local saved path
        lambda_file_path = f"/tmp/{original_solo_file}" 
        
        file_response = s3.get_object(Bucket=bucket, Key=chunk)
        file_content = file_response['Body'].read()
        
        with open(lambda_file_path, 'wb') as f:
            f.write(file_content)
        
        with open(lambda_file_path, 'rb') as audio_file:
            transcript = openai.Audio.transcribe(
                model="whisper-1",
                file=audio_file,
                language="es"
            )
            
        text=text+transcript.text
        
        print(f"{chunk} : \n{transcript}")
    
    
    #upload transcript to s3
    working_file_key=Path(working_file).stem
    file_name=f"{email}/{working_file_key}-transcript.txt"
    s3 = boto3.resource("s3")
    ContentType="text/html; charset=utf-8"
    s3.Bucket(bucket).put_object(Key=file_name, Body=text, ContentType=ContentType)
        
    return {
        'statusCode': 200,
        'body': "End of whisper function!",
        'email':email,
        "input_file": event["input_file"],
        'working_file':working_file,
        'chunks_keys':chunks_keys,
        'transcription_key':file_name,
        'transcript':text
    }

## 5. summarization_version1: Summarize transcript with OpenAi model

External layers required:

Openai: openai_installed_vscode_openai0.26_urllib3 1.26:
      
    pip install --target=C:\Users\priet\projects\aws\packages\openai openai urllib3==1.26
    reasons:https://community.openai.com/t/openai-cannot-import-name-default-ciphers-problem-breaking-changes-with-the-latest-version-of-requests/194265   

import json
import boto3
import os
import math
import re
import string
import time
from pathlib import Path #library to extract name of the file
import openai
from concurrent.futures import ThreadPoolExecutor


def lambda_handler(event, context):
    
    '''
    input={
        "statusCode": 200,
        "body": "End of whisper function!",,
        "email": "###",
        "input_file": "###",
        'working_file': "###",
        'chunks_keys':{
            "###",
              "###"
              },
        "transcription_key": "###",
        "transcript": "text..."
    }
    
    '''
    input=event
    
    print(f"input : {input}")
    
    
    bucket="###"
    
    #1. Get the info from the event
    print("#1. Get the info from the event")
    
    #main inputs. chunk_key is a str 
    
    email=input["email"]
    input_file=input["input_file"]
    working_file=input["working_file"]
    chunks_keys=input["chunks_keys"]
    transcription_key=input["transcription_key"]
    
    
    print(transcription_key)
    
    s3 = boto3.client("s3")
    file = s3.get_object(Bucket=bucket, Key=transcription_key)
    print(file)
    text=(file["Body"]).read().decode('utf-8')
    print(f"\n {text}")
    
        
    transcript_text=text
    transcript_text_lenght=len(transcript_text)
    print(f"text had {transcript_text_lenght} words")
    
    
    #2. divide the text by complete words and not by character, stored all words into an array
    print("\n #2. divide the text by complete words and not by character, stored all words into an array")
    words=re.split('['+string.punctuation+']', transcript_text)
    print(words)
    
    #defines the instruction context to pass along each chunk to GPT model for summary of each chunk
    i=1
    text_chunks=0
    instruction = "Taking the following text into account, write an exhaustive summary that addresses the topics discussed, the decisions agreed upon, the positions and opinions expressed, and the commitments agreed between the participants. The following text is only a part of a whole : " 
    print(instruction)
    
    
    #3. creates a function to count tokens of a text
    print("\n #3. creates a function to count tokens of a text")
    
    
    #based on my the code in my local machine. instruction has 55 tokens. however has 34 words. thus relation betweem words and tokens is 1.617 tokens per word
    instruction_words=272
    instruction_tokens=84
    print(f"\n La instruccion tiene {instruction_words} words")
    print(f"\n La instruccion tiene {instruction_tokens} tokens")
    
    
    
    #4. Takes the num_tokens_from_string function to calculate text tokens and how many chunks are requited to query GPT models
    #due to the tokenizationfuncion is not working beacuse dependecies errors, this estimation is based on results in my local machine. All estimation is based on words not tokens.
    
    print("\n #4. Takes the num_tokens_from_string function to calculate text tokens and how many chunks are requited to query GPT models")
    
    relation_words_tokens=0.31

    print(f"\nText has {transcript_text_lenght} words")
    
    
    text_tokens=transcript_text_lenght*relation_words_tokens
    print(f"\nText has {text_tokens} tokens")
    
    
    chunk_limit_tokens=4000*0.85 #defined for text-davinci-002 at 85%
    
    chunk_limit_words=chunk_limit_tokens/relation_words_tokens
    prompt_limit_tokens=0.8*chunk_limit_tokens #define that 80% of the space will be occupy by the prompt in tokens
    prompt_limit_words=0.8*chunk_limit_words #define that 80% of the space will be occupy by the prompt in words
    print(f"\nPrompt must not be over {prompt_limit_words} words")
    
    chunk_size=math.floor(prompt_limit_words-instruction_words)
    
    text_chunks=math.ceil(transcript_text_lenght/chunk_size)
    
    print(f"\nText will be slice in {text_chunks} chunks, each one with {chunk_size} characters (no incl. instruction)")
    
    
    #find out the cost of the summarization
    cost_per_1k_tokens=0.00200 #gpt4
    cost_per_1k_tokens=0.0015 #gpt-3.5-turbo
    total_tokens=text_tokens+(instruction_tokens*chunk_size)
    summarization_cost=total_tokens*cost_per_1k_tokens/1000
    print(f"\nAprox. costo of summarization is {round(summarization_cost,2)} USD")
    
    
    print(f"Words lengh are: {len(words)}")

    
    #create chunks based on the number of parts identified and also joining parts together, to comply with the prompt limit: 
   
    temp_prompt=""
    index_prompt=1
    prompts=[]
    i=1
    chunk_size_incl_instruction=chunk_size+instruction_words
    print(chunk_size_incl_instruction)
    
    total_chunks_len=0
    x=len(words)
    while i < x:
    
        #temp_prompt=instruction
        temp_prompt=""
        
    
        while len(temp_prompt)<chunk_size_incl_instruction:
            
            #i-1 because index for list words starts at '0'
            if len(temp_prompt+words[i-1]) > chunk_size_incl_instruction:
                #print(f"  condicion 1 len(temp_prompt+words[i-1]):{len(temp_prompt+words[i-1])}, chunk_size_incl_instruction:{chunk_size_incl_instruction}, i:{i}")
                break
    
            temp_prompt=temp_prompt+words[i-1]
            #print(f"  len temp_prompt {len(temp_prompt)} , i:{i}")
    
            i=i+1
            
            if i>=x: 
                print(f"   condicion 2 i:{i} >= x:{x}")
                break
        
        #print(f"\n{index_prompt}. prompt: \n{temp_prompt}")
        #print(f"{index_prompt}\nchunk_len: {len(temp_prompt)}")
        
        total_chunks_len=total_chunks_len+len(temp_prompt)
        #print(f"total_chunks_len: {total_chunks_len}")
        #print(f"counter i reach {i}\n")
        
        prompts.append(temp_prompt)
    
        #print(f"\n\n{prompts}")
        #print(f"\n\n{len(prompts)}")
        index_prompt=index_prompt+1  
    
    
    #print(f"counter i reach {i}")
    print(f"\n there are {len(prompts)} prommpts")
    
    
    
    answ_limit_words=chunk_limit_words-prompt_limit_words
    
    answ_limit_tokens=chunk_limit_tokens-prompt_limit_tokens
    
    #5. Call GPT Model to summaryze each chunk
    print("#5. Call GPT Model to summaryze each chunk")
    
    openai.api_key=os.environ['OPENAI_API_KEY']
    
    print(f"max len is {chunk_size}")
   
    #allow this code it to execute the calls of OPENAI in parallel, reducing time by 5x
    print(f"prompts are: {prompts}")
    
    def call_openai(args):
        index, prompt = args
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role":"system","content":instruction},
                {"role":"user","content":prompt}
            ],
            temperature=0,
            max_tokens=int(answ_limit_tokens),
            top_p=1.0,
            frequency_penalty=0,
            presence_penalty=1
        )
    
        #response_text=a['choices'][0]['text']
        response_text=response['choices'][0]['message']['content']
    
        return (index,response_text)
    
    # Use ThreadPoolExecutor to execute multiple API calls concurrently
    with ThreadPoolExecutor() as executor:
        # Use enumerate to provide both index and prompt in the input for call_openai
        results = list(executor.map(call_openai, enumerate(prompts)))

    # Sort the results by index and extract the response_text
    sorted_results = [response_text for _, response_text in sorted(results, key=lambda x: x[0])]
    
    partial_full_summary="\n".join(sorted_results)
    print(f"Partial Summary in list format is {partial_full_summary}")
    
    partial_full_summary=sorted_results
    print(f"\nPartial Summary in list format and after organize is \n {partial_full_summary}")
    
    partial_full_summary="\n\n".join(partial_full_summary)
    print(f"Partial Summary in str format is \n {partial_full_summary}")
   
    full_summary=partial_full_summary
    
    #prints the whole summary   
    print("\n")
    print(f"\nFull summary:\n\n {full_summary}")
    
    
    #print various 
    print(f"Summary has{len(full_summary)} characters")
    
    
    # 6.create a summarization text file inside s3
    print("# 6.create a summarization text file inside s3")
     
    working_file_key=Path(working_file).stem
    file_name=f"{email}/{working_file_key}-summarization.txt"
    s3 = boto3.resource("s3")
    ContentType="text/html; charset=utf-8"
    s3.Bucket(bucket).put_object(Key=file_name, Body=full_summary, ContentType=ContentType)
    
    return {
        'statusCode': 200,
        'body': 'End of summarization function!',
        'email':email,
        'input_file':event["input_file"],
        'working_file':working_file,
        'chunks_keys':chunks_keys,
        "transcription_key":transcription_key,
        'transcript_text':transcript_text,
        'summary_key':file_name,
        'summary':full_summary
    }
    
    

## 6. results_TranscriptionAI_Version1 (consolidate all info, create html document for each product, eliminate video/audio files, send email to user and send data to Dynamo DB.

External layers: N/A

import json
import boto3
import os
from pathlib import Path
from time import gmtime, strftime

#Copy template HTML for the products to be stored in S3

html1='''
<!doctype html>
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office">

<head>
  <title> Transcribe-AI | Resumen </title>
  <!--[if !mso]><!-->
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <!--<![endif]-->
  <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <style type="text/css">
    #outlook a {
      padding: 0;
    }

    body {
      margin: 0;
      padding: 0;
      -webkit-text-size-adjust: 100%;
      -ms-text-size-adjust: 100%;
    }

    table,
    td {
      border-collapse: collapse;
      mso-table-lspace: 0pt;
      mso-table-rspace: 0pt;
    }

    img {
      border: 0;
      height: auto;
      line-height: 100%;
      outline: none;
      text-decoration: none;
      -ms-interpolation-mode: bicubic;
    }

    p {
      display: block;
      margin: 13px 0;
    }
  </style>
  <!--[if mso]>
        <noscript>
        <xml>
        <o:OfficeDocumentSettings>
          <o:AllowPNG/>
          <o:PixelsPerInch>96</o:PixelsPerInch>
        </o:OfficeDocumentSettings>
        </xml>
        </noscript>
        <![endif]-->
  <!--[if lte mso 11]>
        <style type="text/css">
          .mj-outlook-group-fix { width:100% !important; }
        </style>
        <![endif]-->
  <!--[if !mso]><!-->
  <link href="https://fonts.googleapis.com/css?family=Roboto:300,500" rel="stylesheet" type="text/css">
  <style type="text/css">
    @import url(https://fonts.googleapis.com/css?family=Roboto:300,500);
  </style>
  <!--<![endif]-->
  <style type="text/css">
    @media only screen and (min-width:480px) {
      .mj-column-per-100 {
        width: 100% !important;
        max-width: 100%;
      }

      .mj-column-per-45 {
        width: 45% !important;
        max-width: 45%;
      }
    }
  </style>
  <style media="screen and (min-width:480px)">
    .moz-text-html .mj-column-per-100 {
      width: 100% !important;
      max-width: 100%;
    }

    .moz-text-html .mj-column-per-45 {
      width: 45% !important;
      max-width: 45%;
    }
  </style>
  <style type="text/css">
    @media only screen and (max-width:480px) {
      table.mj-full-width-mobile {
        width: 100% !important;
      }

      td.mj-full-width-mobile {
        width: auto !important;
      }
    }
  </style>
</head>

<body style="word-spacing:normal;">
  <div style="">
    <!--[if mso | IE]><table align="center" border="0" cellpadding="0" cellspacing="0" class="" style="width:600px;" width="600" ><tr><td style="line-height:0px;font-size:0px;mso-line-height-rule:exactly;"><![endif]-->
    <div style="margin:0px auto;max-width:600px;">
      <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="width:100%;">
        <tbody>
          <tr>
            <td style="direction:ltr;font-size:0px;padding:0px;text-align:center;">
              <!--[if mso | IE]><table role="presentation" border="0" cellpadding="0" cellspacing="0"><tr><td class="" style="vertical-align:top;width:600px;" ><![endif]-->
              <div class="mj-column-per-100 mj-outlook-group-fix" style="font-size:0px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;">
                <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="vertical-align:top;" width="100%">
                  <tbody>
                    <tr>
                      <td align="center" style="font-size:0px;padding:10px 25px;word-break:break-word;">
                        <div style="font-family:Roboto, Helvetica, sans-serif;font-size:10px;font-weight:300;line-height:24px;text-align:center;color:#616161;">Aumente la productividad con Transcribe-AI</div>
                      </td>
                    </tr>
                  </tbody>
                </table>
              </div>
              <!--[if mso | IE]></td></tr></table><![endif]-->
            </td>
          </tr>
        </tbody>
      </table>
    </div>
    <!--[if mso | IE]></td></tr></table><table align="center" border="0" cellpadding="0" cellspacing="0" class="" style="width:600px;" width="600" ><tr><td style="line-height:0px;font-size:0px;mso-line-height-rule:exactly;"><![endif]-->
    <div style="margin:0px auto;max-width:600px;">
      <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="width:100%;">
        <tbody>
          <tr>
            <td style="direction:ltr;font-size:0px;padding:0px;text-align:center;">
              <!--[if mso | IE]><table role="presentation" border="0" cellpadding="0" cellspacing="0"><tr><td class="" style="vertical-align:top;width:600px;" ><![endif]-->
              <div class="mj-column-per-100 mj-outlook-group-fix" style="font-size:0px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;">
                <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="vertical-align:top;" width="100%">
                  <tbody>
                    <tr>
                      <td align="center" style="font-size:0px;padding:10px 25px;word-break:break-word;">
                        <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="border-collapse:collapse;border-spacing:0px;">
                          <tbody>
                            <tr>
                              <td style="width:550px;">
                                <a href="https://www.transcribe-ai.com/" target="_blank">
                                  <img height="auto" src="https://media.istockphoto.com/id/1220573371/photo/minimal-work-space-creative-flat-lay-photo-of-workspace-desk-top-view-office-desk-with-laptop.jpg?s=612x612&w=0&k=20&c=xmrSzPD4LCRhPD4L5TlttC88sSYe9Pc3J2ZnxCMzzyQ=" style="border:0;display:block;outline:none;text-decoration:none;height:auto;width:100%;font-size:13px;" width="550" />
                                </a>
                              </td>
                            </tr>
                          </tbody>
                        </table>
                      </td>
                    </tr>
                  </tbody>
                </table>
              </div>
              <!--[if mso | IE]></td></tr></table><![endif]-->
            </td>
          </tr>
        </tbody>
      </table>
    </div>
    <!--[if mso | IE]></td></tr></table><table align="center" border="0" cellpadding="0" cellspacing="0" class="" style="width:600px;" width="600" ><tr><td style="line-height:0px;font-size:0px;mso-line-height-rule:exactly;"><![endif]-->
    <div style="margin:0px auto;max-width:600px;">
      <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="width:100%;">
        <tbody>
          <tr>
            <td style="direction:ltr;font-size:0px;padding:0px;text-align:center;">
              <!--[if mso | IE]><table role="presentation" border="0" cellpadding="0" cellspacing="0"><tr><td class="" style="vertical-align:top;width:600px;" ><![endif]-->
              <div class="mj-column-per-100 mj-outlook-group-fix" style="font-size:0px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;">
                <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="vertical-align:top;" width="100%">
                  <tbody>
                    <tr>
                      <td align="center" style="font-size:0px;padding:10px 25px;word-break:break-word;">
                        <div style="font-family:Roboto, Helvetica, sans-serif;font-size:16px;font-weight:300;line-height:24px;text-align:center;color:#616161;"></div>
                      </td>
                    </tr>
                  </tbody>
                </table>
              </div>
              <!--[if mso | IE]></td></tr></table><![endif]-->
            </td>
          </tr>
        </tbody>
      </table>
    </div>
    <!--[if mso | IE]></td></tr></table><table align="center" border="0" cellpadding="0" cellspacing="0" class="" style="width:600px;" width="600" ><tr><td style="line-height:0px;font-size:0px;mso-line-height-rule:exactly;"><![endif]-->
    <div style="margin:0px auto;max-width:600px;">
      <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="width:100%;">
        <tbody>
          <tr>
            <td style="direction:ltr;font-size:0px;padding:20px 0;text-align:center;">
              <!--[if mso | IE]><table role="presentation" border="0" cellpadding="0" cellspacing="0"><tr><td class="" style="vertical-align:top;width:270px;" ><![endif]-->
              <div class="mj-column-per-45 mj-outlook-group-fix" style="font-size:0px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;">
                <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="vertical-align:top;" width="100%">
                  <tbody>
                    <tr>
                      <td align="center" style="font-size:0px;padding:0px;padding-bottom:0px;word-break:break-word;">
                        <div style="font-family:Roboto, Helvetica, sans-serif;font-size:16px;font-weight:300;line-height:24px;text-align:center;color:#616161;">
                          <h3 style="font-weight: bold; color: #3498DB; text-decoration: none;">
'''

html2='''
</h3>
                        </div>
                      </td>
                    </tr>
                    <tr>
                      <td align="center" style="font-size:0px;padding:10px 25px;word-break:break-word;">
                        <p style="border-top:solid 2px #616161;font-size:1px;margin:0px auto;width:100%;">
                        </p>
                        <!--[if mso | IE]><table align="center" border="0" cellpadding="0" cellspacing="0" style="border-top:solid 2px #616161;font-size:1px;margin:0px auto;width:220px;" role="presentation" width="220px" ><tr><td style="height:0;line-height:0;"> &nbsp;
</td></tr></table><![endif]-->
                      </td>
                    </tr>
                    <tr>
                      <td align="center" style="font-size:0px;padding:10px 25px;word-break:break-word;">
                        <p style="border-top:solid 2px #616161;font-size:1px;margin:0px auto;width:45%;">
                        </p>
                        <!--[if mso | IE]><table align="center" border="0" cellpadding="0" cellspacing="0" style="border-top:solid 2px #616161;font-size:1px;margin:0px auto;width:99px;" role="presentation" width="99px" ><tr><td style="height:0;line-height:0;"> &nbsp;
</td></tr></table><![endif]-->
                      </td>
                    </tr>
                  </tbody>
                </table>
              </div>
              <!--[if mso | IE]></td></tr></table><![endif]-->
            </td>
          </tr>
        </tbody>
      </table>
    </div>
    <!--[if mso | IE]></td></tr></table><table align="center" border="0" cellpadding="0" cellspacing="0" class="" style="width:600px;" width="600" ><tr><td style="line-height:0px;font-size:0px;mso-line-height-rule:exactly;"><![endif]-->
    <div style="margin:0px auto;max-width:600px;">
      <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="width:100%;">
        <tbody>
          <tr>
            <td style="direction:ltr;font-size:0px;padding:20px 0;padding-top:0px;text-align:center;">
              <!--[if mso | IE]><table role="presentation" border="0" cellpadding="0" cellspacing="0"><tr><td class="" style="vertical-align:top;width:600px;" ><![endif]-->
              <div class="mj-column-per-100 mj-outlook-group-fix" style="font-size:0px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;">
                <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="vertical-align:top;" width="100%">
                  <tbody>
                    <tr>
                      <td align="left" style="font-size:0px;padding:10px 25px;word-break:break-word;">
                        <div style="font-family:Roboto, Helvetica, sans-serif;font-size:16px;font-weight:300;line-height:24px;text-align:left;color:#616161;">
                          <p style="text-align: justify;">
'''

html3='''
</p>
                          <p></p>
                        </div>
                      </td>
                    </tr>
                  </tbody>
                </table>
              </div>
              <!--[if mso | IE]></td></tr></table><![endif]-->
            </td>
          </tr>
        </tbody>
      </table>
    </div>
    <!--[if mso | IE]></td></tr></table><![endif]-->
    <p style="border-top:solid 1px #E0E0E0;font-size:1px;margin:0px auto;width:100%;">
    </p>
    <!--[if mso | IE]><table align="center" border="0" cellpadding="0" cellspacing="0" style="border-top:solid 1px #E0E0E0;font-size:1px;margin:0px auto;width:550px;" role="presentation" width="550px" ><tr><td style="height:0;line-height:0;"> &nbsp;
</td></tr></table><table align="center" border="0" cellpadding="0" cellspacing="0" class="" style="width:600px;" width="600" ><tr><td style="line-height:0px;font-size:0px;mso-line-height-rule:exactly;"><![endif]-->
    <div style="margin:0px auto;max-width:600px;">
      <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="width:100%;">
        <tbody>
          <tr>
            <td style="direction:ltr;font-size:0px;padding:0px;text-align:center;">
              <!--[if mso | IE]><table role="presentation" border="0" cellpadding="0" cellspacing="0"><tr><td class="" style="vertical-align:top;width:600px;" ><![endif]-->
              <div class="mj-column-per-100 mj-outlook-group-fix" style="font-size:0px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;">
                <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="vertical-align:top;" width="100%">
                  <tbody>
                    <tr>
                      <td align="center" style="font-size:0px;padding:10px 25px;word-break:break-word;">
                        <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="border-collapse:collapse;border-spacing:0px;">
                          <tbody>
                            <tr>
                              <td style="width:25px;">
                                <a href="https://www.transcribe-ai.com/" target="_blank">
                                  <img height="auto" src="https://www.transcribe-ai.com/files/logo.svg" style="border:0;display:block;outline:none;text-decoration:none;height:auto;width:100%;font-size:13px;" width="25" />
                                </a>
                              </td>
                            </tr>
                          </tbody>
                        </table>
                      </td>
                    </tr>
                  </tbody>
                </table>
              </div>
              <!--[if mso | IE]></td></tr></table><![endif]-->
            </td>
          </tr>
        </tbody>
      </table>
    </div>
    <!--[if mso | IE]></td></tr></table><![endif]-->
  </div>
</body>

</html>
'''


def lambda_handler(event, context):
    
    '''
    {
        "statusCode": 200,
        "body": "End of function!",
        "email": "###",
        "input_file":"###",
        "working_file": "###",
        "chunks_keys": [
            "###",
            "###"
          ],
        "transcription_key":"###",
        "transcript_text": "Buenas ...",
        "summary_key": "###",
        "summary": "\n\nEn la reunión s..."
    }
    '''
    input=event
    
    print(f"event is {event}")
    
    email=input["email"]
    input_file=input["input_file"]
    working_file=input["working_file"]
    chunks_keys=input["chunks_keys"]
    transcription_key=input["transcription_key"]
    transcript_text=input["transcript_text"]
    summary_key=input["summary_key"]
    summary=input["summary"]
    
    bucketName = os.environ.get("bucketName")
    
    #1. Create a HTML of each product.
    print("#1. Create a HTML of each product.")
    
    s3 = boto3.resource("s3")
    working_file_key=Path(working_file).stem
    
    #format summary text as html to keep row spaces
    summary_html_text=summary
    summary_html_text = summary_html_text.replace("\n", "<br/>")
    summary_html_text = summary_html_text.replace("\t", "&nbsp;&nbsp;&nbsp;&nbsp;")
    
    #save summary in html template
    html_summary=f"{html1}RESUMEN{html2}{summary_html_text}{html3}"
    summary_key_html=f"{email}/{working_file_key}-summarization.html"
    ContentType="text/html; charset=utf-8"
    s3.Bucket(bucketName).put_object(Key=summary_key_html, Body=html_summary, ContentType=ContentType)
    
    #format transcript text as html to keep row spaces
    transcript_html_text=transcript_text
    transcript_html_text = transcript_html_text.replace("\n", "<br/>")
    transcript_html_text = transcript_html_text.replace("\t", "&nbsp;&nbsp;&nbsp;&nbsp;")
    
    #save transcript in html template
    html_transcript=f"{html1}TRANSCRIPCION{html2}{transcript_html_text}{html3}"
    transcription_key_html=f"{email}/{working_file_key}-transcription.html"
    ContentType="text/html; charset=utf-8"
    s3.Bucket(bucketName).put_object(Key=transcription_key_html, Body=html_transcript, ContentType=ContentType)
    
    
    #2. get presigned URl for summary and transcription
    print("#2. get presigned URl for summary.html and transcription.html")
    
    s3 = boto3.client("s3")
    
    print(bucketName)
    
    #expiration url set to 1 week.
    expiration=604800
    
    #get the url for the summary document
    print(summary_key)
    summary_presign_url= s3.generate_presigned_url(
        'get_object',
        Params={'Bucket': bucketName, 'Key': summary_key_html},
        ExpiresIn=expiration)
    print(summary_presign_url)
    
    #get the url for the transcription document
    print(transcription_key)
    transcription_presign_url= s3.generate_presigned_url(
        'get_object',
        Params={'Bucket': bucketName, 'Key': transcription_key_html},
        ExpiresIn=expiration)
    print(transcription_presign_url)
    
    
    #3. Eliminate all audio files inside s3 bucket input file, working file, chunks_keys
    print("#3. Eliminate all audio files inside s3 bucket input file, working file, chunks_keys")
    
    files_to_delete=[]
    file_input_path=f"{email}/{input_file}"
    working_input_path=f"{email}/{working_file}"
    
    if input_file == working_file:
            files_to_delete.append(file_input_path)
            files_to_delete= files_to_delete + chunks_keys
    else:
        files_to_delete.append(file_input_path)
        files_to_delete.append(working_input_path)
        files_to_delete= files_to_delete + chunks_keys   
    
    for file in files_to_delete:
        print(f"deleting file:{str(file)}")
        s3.delete_object(
            Bucket=bucketName,
            Key=str(file)
        )
    
    #4. send email through SES
    print("#4. send email through SES")
    
    client = boto3.client("ses", region_name="us-east-1")
    
    #use template emails to send the result to 
    response = client.send_templated_email(
        Source = "###@###.com", 
        Destination = {
            "ToAddresses": [email] 
        },
        ReplyToAddresses=[email],
        Template="###",
        TemplateData=json.dumps({
            "working_file":working_file,
            "transcription_key":transcription_presign_url,
            "summary_key":summary_presign_url
        })
    )
    
    #5. Update info in DynamoDB table (###)
    print("#5. Update info in DynamoDB table (###)")
    
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('###')
    
    # store the current time in a human readable format in a variable
    date= strftime("%d %b %Y")
    time= strftime(" %R ")
    
    
    #update table in dynemodb. 
    response2=table.update_item(
        Key={"ID":input_file},
        UpdateExpression='SET working_file = :working_file, processed_date = :processed_date, processed_time = :processed_time, transcription_key = :transcription_key, transcription_key_html = :transcription_key_html, transcript_text = :transcript_text, summary_key = :summary_key, summary_key_html = :summary_key_html, summary = :summary',
        ExpressionAttributeValues={
            ":working_file": working_file,
            ":processed_date": date,
            ":processed_time":time,
            ":transcription_key":transcription_key,
            ":transcription_key_html":transcription_key_html,
            ":transcript_text":transcript_text,
            ":summary_key": summary_key,
            ":summary_key_html":summary_key_html,
            ":summary": summary
        },
    )
    
    print(response2)
    
    
    return {
        "statusCode": 200,
        "body" : "End of process transcription + summarization process",
        "email": email,
        "input_file":input_file,
        "working_file": working_file,
        "transcription_key":transcription_key,
        "transcript_text":transcript_text,
        "summary_key": summary_key,
        "summary": summary
    }
